# SA-Excited states with DMRG + NWChem + Orbital refinement

### Parameters

In [37]:
from pyscf import gto, scf
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf, ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)

ket = driver.get_random_mps(tag="KET", bond_dim=100, nroots=3)
energies = driver.dmrg(mpo, ket, n_sweeps=10, bond_dims=[100], noises=[1e-5] * 4 + [0], thrds=[1e-10] * 8, iprint=1)

print('State-averaged MPS energies = [%s]' % " ".join("%20.15f" % x for x in energies))


Sweep =    0 | Direction =  forward | Bond dimension =  100 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      0.224 | E[  3] =    -107.6541224072   -106.9596261045   -106.9437562448 | DW = 3.56574e-07

Sweep =    1 | Direction = backward | Bond dimension =  100 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      0.311 | E[  3] =    -107.6541224475   -106.9596261547   -106.9437569389 | DE = -6.94e-07 | DW = 7.66385e-08

Sweep =    2 | Direction =  forward | Bond dimension =  100 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      0.409 | E[  3] =    -107.6541224475   -106.9596261547   -106.9437569389 | DE = -3.81e-12 | DW = 3.22858e-07

Sweep =    3 | Direction = backward | Bond dimension =  100 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      0.511 | E[  3] =    -107.6541224475   -106.9596261547   -106.9437569389 | DE = -1.36e-11 | DW = 7.66379e-08

Sweep =    4 | Direction =  forward | Bond dimension =  100 | Noise 

In [38]:
kets = [driver.split_mps(ket, ir, tag="KET-%d" % ir) for ir in range(ket.nroots)]
sa_1pdm = np.mean([driver.get_1pdm(k) for k in kets], axis=0)
sa_2pdm = np.mean([driver.get_2pdm(k) for k in kets], axis=0).transpose(0,3,1,2)


print('Energy from pdms = %20.15f' % (np.einsum('ij,ij->', sa_1pdm, h1e) + 0.5 * np.einsum('ijkl,ijkl->', sa_2pdm, driver.unpack_g2e(g2e)) + ecore))


Energy from pdms = -107.185831499712961
